In [47]:
import pandas as pd
import numpy as np

Read SPEW synthesis data

In [48]:
data_dir = '../spew_data'
data_prex = '2010_ver1_45079_synth_'
pdata_file = data_dir + '/' + data_prex + 'people.txt'
hdata_file = data_dir + '/' + data_prex + 'households.txt'

In [49]:
pdata = pd.read_csv(pdata_file, dtype = {'sp_id': object, 'sp_hh_id': object})
hdata = pd.read_csv(hdata_file, dtype = {'sp_id': object})

In [50]:
pdata.columns

Index(['sp_id', 'sp_hh_id', 'serialno', 'stcotrbg', 'age', 'sex', 'race',
       'sporder', 'relate', 'sp_school_id', 'sp_work_id'],
      dtype='object')

In [51]:
hdata.columns

Index(['sp_id', 'serialno', 'stcotrbg', 'hh_race', 'hh_income', 'hh_size',
       'hh_age', 'latitude', 'longitude'],
      dtype='object')

In [52]:
pdata = pdata.drop(columns=['serialno', 'stcotrbg', 'sp_school_id', 'sp_work_id'])

In [53]:
hdata = hdata[['sp_id', 'hh_income']]

In [54]:
phdata = pd.merge(pdata, hdata, left_on=['sp_hh_id'], right_on = ['sp_id'])

In [55]:
phdata

,sp_id_x,sp_hh_id,age,sex,race,sporder,relate,sp_id_y,hh_income
0,164099532,48930706,49,2,2,1,0,48930706,115000
1,164099533,48930706,57,1,2,2,1,48930706,115000
2,164099534,48954569,49,2,2,1,0,48954569,115000
3,164099535,48954569,57,1,2,2,1,48954569,115000
4,164099538,48964134,49,2,2,1,0,48964134,115000
...,...,...,...,...,...,...,...,...,...
352053,164066165,48841329,50,1,1,1,0,48841329,90000
352054,164066169,48822193,50,1,1,1,0,48822193,90000
352055,164066170,48815049,50,1,1,1,0,48815049,90000
352056,164066171,48811092,50,1,1,1,0,48811092,90000


In [56]:
cdn_data = phdata[phdata['age'] < 18]

In [57]:
cdn_data

,sp_id_x,sp_hh_id,age,sex,race,sporder,relate,sp_id_y,hh_income
102,164426008,48996117,4,1,2,3,2,48996117,97000
103,164426009,48996117,0,1,2,4,2,48996117,97000
106,164426012,49005231,4,1,2,3,2,49005231,97000
107,164426013,49005231,0,1,2,4,2,49005231,97000
110,164426016,48930748,4,1,2,3,2,48930748,97000
...,...,...,...,...,...,...,...,...,...
352036,163958884,48979669,3,1,1,2,2,48979669,42000
352038,163971928,48840456,5,1,2,2,2,48840456,46420
352041,163971943,48804862,5,1,2,2,2,48804862,46420
352044,163971964,48836724,5,1,2,2,2,48836724,46420


In [58]:
cdn_data.keys()

Index(['sp_id_x', 'sp_hh_id', 'age', 'sex', 'race', 'sporder', 'relate',
       'sp_id_y', 'hh_income'],
      dtype='object')

In [59]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

BRFSS distribution of ACEs according to house hold income

In [60]:
race_list

{1: 'White', 2: 'Black', 3: 'Hispanic', 4: 'Other', 5: 'Multi'}

In [61]:
income_list

{1: '< 15000',
 2: '15000 - 24999',
 3: '25000 - 34999',
 4: '35000 - 49999',
 5: '50000 +',
 9: "Don't Know"}

In [62]:
ace_list

{1: 'depress',
 2: 'alcoabuse',
 3: 'drugabuse',
 4: 'prison',
 5: 'patdivorce',
 6: 'phyabuse1',
 7: 'phyabuse2',
 8: 'verbalabuse',
 9: 'sexabuse1',
 10: 'sexabuse2',
 11: 'sexabuse3'}

In [63]:
df

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.211705        0.010981  0.190183  0.233228
                15000 - 24999    0.163790        0.007248  0.149584  0.177996
                25000 - 34999    0.146769        0.008281  0.130538  0.163000
                35000 - 49999    0.153333        0.006934  0.139743  0.166924
                50000 +          0.141829        0.003853  0.134278  0.149380
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.166667        0.036901  0.094343  0.238991
                25000 - 34999    0.060000        0.033586 -0.005827  0.125827
                35000 - 49999    0.059524        0.025815  0.008926  0.110121
                50000 +          0.054264        0.019945  0.015171  0.093356
                Don't Know       0.120000        0.037523  0.046456  0.193544

[330 rows x 4 columns]

In [64]:
df = df.sort_index(); df

Proportion  Standard Error  L 95% CI  \
alcoabuse   Black 15000 - 24999    0.218512        0.010002  0.198909   
                  25000 - 34999    0.208817        0.013844  0.181683   
                  35000 - 49999    0.209190        0.014143  0.181469   
                  50000 +          0.207373        0.011236  0.185351   
                  < 15000          0.219140        0.010893  0.197789   
...                                     ...             ...       ...   
verbalabuse White 25000 - 34999    0.267847        0.010418  0.247429   
                  35000 - 49999    0.274701        0.008635  0.257776   
                  50000 +          0.276256        0.004956  0.266543   
                  < 15000          0.345848        0.012781  0.320799   
                  Don't Know       0.212340        0.007409  0.197819   

                                 U 95% CI  
alcoabuse   Black 15000 - 24999  0.238115  
                  25000 - 34999  0.235951  
                  35000 - 49999  0.236910  
                  50000 +        0.229395  
                  < 15000        0.240491  
...                                   ...  
verbalabuse White 25000 - 34999  0.288265  
                  35000 - 49999  0.291625  
                  50000 +        0.285969  
                  < 15000        0.370898  
                  Don't Know     0.226861  

[330 rows x 4 columns]

In [65]:
income_cat = [0, 15000, 24999, 34999, 49999]

race_cat = [1,2,3,4,5]
ace_dist = {1:[0.2117052, 0.1637898, 0.1467698, 0.1533333, 0.1418293, 0.1085915], 
            2:[0.0834492, 0.0799530, 0.0733411, 0.0677146, 0.0846154, 0.0606601], 
            3:[0.1746032, 0.0763359, 0.1388889, 0.0909091, 0.1901408, 0.2162162], 
            4:[0.2500000, 0.1785714, 0.2033898, 0.1836735, 0.0958904, 0.1282051], 
            5:[0.2638889, 0.2857143, 0.3529412, 0.1785714, 0.2692308, 0.2266667]}

def cat_income(hh_income):
    if pd.isnull(hh_income):
        return 9
    if hh_income >= income_cat[0] and hh_income < income_cat[1]:
        return 1
    if hh_income >= income_cat[1] and hh_income < income_cat[2]:
        return 2
    if hh_income >= income_cat[2] and hh_income < income_cat[3]:
        return 3
    if hh_income >= income_cat[3] and hh_income < income_cat[4]:
        return 4
    if hh_income >= income_cat[4]:
        return 5

def cat_race(chd_race):
    if pd.isnull(chd_race):
        return 4
    if chd_race == 1:
        return 1
    if chd_race == 2:
        return 2
    if chd_race == 9:
        return 5
    return 4
    

In [66]:
def output_model(model):
    # agents = [a.output() 
    #          for i, sch_list in model.sch_group.items()
    #          for scheduler in sch_list 
    #          for a in scheduler.agents ]
    agents = [a.output() for a in model.schedule.agents]
    return pd.DataFrame(agents, columns=['id', 'age', 'race', 'income'] + list(ace_list.values()))
    # return agents

In [67]:
class Children(Agent):
            
    def __init__(self, chd_info, model, pos = None):
        unique_id = chd_info['sp_id_x']
        super().__init__(unique_id, model)
        self.pos = pos
        # self.ace = 0
        self.age = chd_info['age']
        self.sex = chd_info['sex']
        self.income = chd_info['hh_income']
        self.race = chd_info['race']
        self.aces = {key:0 for key in ace_list.keys()}
        
    def step(self):
        r_code = race_list[cat_race(self.race)]
        i_code = income_list[cat_income(self.income)]
        dist_df = df.loc[list(ace_list.values()), r_code, i_code]
        for ace_key in ace_list.keys():
            p_aces = self.model.random.random()
            dist_cat = dist_df.loc[ace_list[ace_key], r_code, i_code]['Proportion']
            if p_aces < dist_cat:
                self.aces[ace_key] = 1
            else:
                self.aces[ace_key] = 0
            
    def get_cat(self):
        return cat_race(self.race), cat_income(self.income)
    
    def output(self):
        return [self.unique_id, self.age, self.race, self.income, ] + list(self.aces.values())

In [68]:
class AceModel(Model):
    def __init__(self, chd_data):
        self.num_agents = len(chd_data)
        self.schedule = RandomActivation(self)
        self.sch_group = {x: [RandomActivation(self) 
                              for i in range(len(income_cat) + 1)] 
                          for x in race_cat}
        
        for i,chd_info in chd_data.iterrows():
            a = Children(chd_info, self)
            cat_r, cat_i = a.get_cat()
            self.schedule.add(a)
            self.sch_group[cat_r][cat_i].add(a)
            
        self.datacollector = DataCollector(
            model_reporters = {'Output': output_model}            
        )
        
    def step(self):        
        for i, sch_list in self.sch_group.items():
            for scheduler in sch_list:
                self.reset_randomizer()
                scheduler.step()
        self.datacollector.collect(self)
        # self.schedule.step()

In [69]:
acemodel = AceModel(cdn_data)

In [70]:
acemodel.step()

In [71]:
res =acemodel.datacollector.model_vars

In [72]:
res = res['Output']; res

[              id  age  race  income  depress  alcoabuse  drugabuse  prison  \
 0      164426008    4     2   97000        0          0          0       0   
 1      164426009    0     2   97000        0          0          0       0   
 2      164426012    4     2   97000        0          1          0       0   
 3      164426013    0     2   97000        0          1          0       0   
 4      164426016    4     2   97000        0          0          0       0   
 ...          ...  ...   ...     ...      ...        ...        ...     ...   
 90309  163958884    3     1   42000        0          0          0       0   
 90310  163971928    5     2   46420        0          1          0       0   
 90311  163971943    5     2   46420        0          1          0       0   
 90312  163971964    5     2   46420        0          0          0       0   
 90313  163971973    5     2   46420        0          0          0       0   
 
        patdivorce  phyabuse1  phyabuse2  verbalab

In [73]:
test = res[-1]; test

,id,age,race,income,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3
0,164426008,4,2,97000,0,0,0,0,1,0,0,0,0,0,0
1,164426009,0,2,97000,0,0,0,0,1,0,0,1,0,0,0
2,164426012,4,2,97000,0,1,0,0,0,0,0,1,0,1,0
3,164426013,0,2,97000,0,1,0,0,0,1,0,0,0,0,0
4,164426016,4,2,97000,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90309,163958884,3,1,42000,0,0,0,0,0,0,0,1,0,0,0
90310,163971928,5,2,46420,0,1,0,0,0,0,0,0,0,0,0
90311,163971943,5,2,46420,0,1,0,0,1,1,0,0,0,0,0
90312,163971964,5,2,46420,0,0,0,0,0,0,0,0,0,0,0


In [75]:
test['race'] = test['race'].map(cat_race)
test['race'].unique()

array([2, 1, 4, 5])

In [76]:
test['income'] = test['income'].map(cat_income)
test['income'].unique()

array([5, 1, 4, 3, 2])

In [77]:
test

,id,age,race,income,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3
0,164426008,4,2,5,0,0,0,0,1,0,0,0,0,0,0
1,164426009,0,2,5,0,0,0,0,1,0,0,1,0,0,0
2,164426012,4,2,5,0,1,0,0,0,0,0,1,0,1,0
3,164426013,0,2,5,0,1,0,0,0,1,0,0,0,0,0
4,164426016,4,2,5,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90309,163958884,3,1,4,0,0,0,0,0,0,0,1,0,0,0
90310,163971928,5,2,4,0,1,0,0,0,0,0,0,0,0,0
90311,163971943,5,2,4,0,1,0,0,1,1,0,0,0,0,0
90312,163971964,5,2,4,0,0,0,0,0,0,0,0,0,0,0


In [161]:
def cal_test_dist(gpdata, ace_type):
    if len(gpdata) == 0:
        return [0,0,0,0]
    prop = (gpdata[ace_type] == 1).sum()/len(gpdata)
    se = math.sqrt(prop*(1-prop)/len(gpdata))
    return [prop, se, prop - norm.ppf(0.975) * se, prop + norm.ppf(0.975)* se]

In [171]:
def gen_dist_df(test_df):
    test_index = [[], [], []]
    test_dist = []
    for ace in ace_list.values():
        for r in race_list.keys():
            for i in income_list.keys():
                gpdata = test_df[(test_df[['race', 'income']] == [r,i]).all(axis=1)]
                # print([r,i], len(gpdata), cal_test_dist(gpdata, ace))
                test_dist.append(cal_test_dist(gpdata, ace))
                test_index[0].append(ace)
                test_index[1].append(race_list[r])
                test_index[2].append(income_list[i])
    # print(res_index)
    return pd.DataFrame(test_dist, columns=['Proportion', 'Standard Error', 
                                     'L 95% CI', 'U 95% CI'], index=test_index)

In [163]:
test_df = pd.DataFrame(test_dist, columns=['Proportion', 'Standard Error', 
                                     'L 95% CI', 'U 95% CI'], index=test_index)
test_df

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.213746        0.010747  0.192681  0.234810
                15000 - 24999    0.149525        0.010030  0.129866  0.169184
                25000 - 34999    0.143983        0.008048  0.128210  0.159757
                35000 - 49999    0.153476        0.006932  0.139891  0.167062
                50000 +          0.146109        0.002085  0.142021  0.150196
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.169133        0.017237  0.135350  0.202916
                25000 - 34999    0.070981        0.011733  0.047985  0.093978
                35000 - 49999    0.055385        0.012688  0.030517  0.080252
                50000 +          0.055715        0.005497  0.044941  0.066489
                Don't Know       0.000000        0.000000  0.000000  0.000000

[330 rows x 4 columns]

In [172]:
gen_dist_df(test)

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.213746        0.010747  0.192681  0.234810
                15000 - 24999    0.149525        0.010030  0.129866  0.169184
                25000 - 34999    0.143983        0.008048  0.128210  0.159757
                35000 - 49999    0.153476        0.006932  0.139891  0.167062
                50000 +          0.146109        0.002085  0.142021  0.150196
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.169133        0.017237  0.135350  0.202916
                25000 - 34999    0.070981        0.011733  0.047985  0.093978
                35000 - 49999    0.055385        0.012688  0.030517  0.080252
                50000 +          0.055715        0.005497  0.044941  0.066489
                Don't Know       0.000000        0.000000  0.000000  0.000000

[330 rows x 4 columns]

In [173]:
df

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.211705        0.010981  0.190183  0.233228
                15000 - 24999    0.163790        0.007248  0.149584  0.177996
                25000 - 34999    0.146769        0.008281  0.130538  0.163000
                35000 - 49999    0.153333        0.006934  0.139743  0.166924
                50000 +          0.141829        0.003853  0.134278  0.149380
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.166667        0.036901  0.094343  0.238991
                25000 - 34999    0.060000        0.033586 -0.005827  0.125827
                35000 - 49999    0.059524        0.025815  0.008926  0.110121
                50000 +          0.054264        0.019945  0.015171  0.093356
                Don't Know       0.120000        0.037523  0.046456  0.193544

[330 rows x 4 columns]

In [174]:
gen_dist_df(test) - df

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.002041       -0.000234  0.002499  0.001582
                15000 - 24999   -0.014264        0.002782 -0.019717 -0.008812
                25000 - 34999   -0.002786       -0.000234 -0.002328 -0.003243
                35000 - 49999    0.000143       -0.000002  0.000148  0.000138
                50000 +          0.004279       -0.001767  0.007743  0.000815
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.002467       -0.019664  0.041007 -0.036074
                25000 - 34999    0.010981       -0.021853  0.053811 -0.031849
                35000 - 49999   -0.004139       -0.013128  0.021591 -0.029869
                50000 +          0.001452       -0.014448  0.029770 -0.026867
                Don't Know      -0.120000       -0.037523 -0.046456 -0.193544

[330 rows x 4 columns]

In [180]:
result_dist = []
for step in range(1):
    acemodel.step()
    test = acemodel.datacollector.model_vars['Output'][-1]
    test['race'] = test['race'].map(cat_race)
    test['income'] = test['income'].map(cat_income)
    result_dist.append(gen_dist_df(test))
avg_dist = sum(result_dist)/len(result_dist)

In [181]:
avg_dist

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.212371        0.010722  0.191356  0.233386
                15000 - 24999    0.160601        0.010327  0.140360  0.180842
                25000 - 34999    0.152916        0.008250  0.136746  0.169087
                35000 - 49999    0.143491        0.006742  0.130277  0.156705
                50000 +          0.147468        0.002093  0.143365  0.151571
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999    0.158562        0.016795  0.125645  0.191480
                25000 - 34999    0.052192        0.010162  0.032274  0.072110
                35000 - 49999    0.076923        0.014781  0.047953  0.105893
                50000 +          0.055141        0.005470  0.044419  0.065863
                Don't Know       0.000000        0.000000  0.000000  0.000000

[330 rows x 4 columns]

In [182]:
avg_dist - df

Proportion  Standard Error  L 95% CI  U 95% CI
depress   White < 15000          0.000666       -0.000259  0.001174  0.000158
                15000 - 24999   -0.003189        0.003079 -0.009223  0.002846
                25000 - 34999    0.006148       -0.000031  0.006208  0.006087
                35000 - 49999   -0.009842       -0.000192 -0.009465 -0.010219
                50000 +          0.005639       -0.001759  0.009087  0.002190
...                                   ...             ...       ...       ...
sexabuse3 Multi 15000 - 24999   -0.008104       -0.020106  0.031302 -0.047511
                25000 - 34999   -0.007808       -0.023423  0.038101 -0.053717
                35000 - 49999    0.017399       -0.011034  0.039026 -0.004228
                50000 +          0.000877       -0.014475  0.029248 -0.027493
                Don't Know      -0.120000       -0.037523 -0.046456 -0.193544

[330 rows x 4 columns]

In [184]:
avg_dist.loc['depress']

Proportion  Standard Error  L 95% CI  U 95% CI
White    < 15000          0.212371        0.010722  0.191356  0.233386
         15000 - 24999    0.160601        0.010327  0.140360  0.180842
         25000 - 34999    0.152916        0.008250  0.136746  0.169087
         35000 - 49999    0.143491        0.006742  0.130277  0.156705
         50000 +          0.147468        0.002093  0.143365  0.151571
         Don't Know       0.000000        0.000000  0.000000  0.000000
Black    < 15000          0.086891        0.003030  0.080953  0.092829
         15000 - 24999    0.083348        0.003665  0.076164  0.090532
         25000 - 34999    0.067150        0.003142  0.060991  0.073309
         35000 - 49999    0.065957        0.002746  0.060575  0.071338
         50000 +          0.083682        0.001990  0.079781  0.087583
         Don't Know       0.000000        0.000000  0.000000  0.000000
Hispanic < 15000          0.000000        0.000000  0.000000  0.000000
         15000 - 24999    0.000000        0.000000  0.000000  0.000000
         25000 - 34999    0.000000        0.000000  0.000000  0.000000
         35000 - 49999    0.000000        0.000000  0.000000  0.000000
         50000 +          0.000000        0.000000  0.000000  0.000000
         Don't Know       0.000000        0.000000  0.000000  0.000000
Other    < 15000          0.237624        0.042352  0.154616  0.320631
         15000 - 24999    0.198488        0.017342  0.164498  0.232477
         25000 - 34999    0.180894        0.017354  0.146881  0.214908
         35000 - 49999    0.215909        0.025323  0.166277  0.265541
         50000 +          0.093856        0.007578  0.079003  0.108708
         Don't Know       0.000000        0.000000  0.000000  0.000000
Multi    < 15000          0.241546        0.029749  0.183238  0.299854
         15000 - 24999    0.285412        0.020765  0.244713  0.326111
         25000 - 34999    0.354906        0.021863  0.312056  0.397756
         35000 - 49999    0.212308        0.022684  0.167848  0.256767
         50000 +          0.278576        0.010744  0.257518  0.299633
         Don't Know       0.000000        0.000000  0.000000  0.000000